In [1]:
%load_ext autoreload
%autoreload 2
from aria.utils import _load_weight
from aria.config import load_model_config
from aria.tokenizer import AbsTokenizer, SeparatedAbsTokenizer
from src.load_aria_weights import get_p2q
import argparse
import torch
from torch.nn.functional import log_softmax
from accelerate import Accelerator
import torch.nn.functional as F

from aria.data.midi import MidiDict

/home/ubuntu/maikyon/miniconda3/envs/p2q-transformer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer.vocab_size

12608

In [7]:
from train_utils import load_config


M_PATH = "./inference/weights/model.safetensors"
M_CONFIG = "medium-stacked"
tokenizer = AbsTokenizer()
tokenizer.vocab_size
tokenizer.add_tokens_to_vocab(['<SEP>'])
config = load_config("train_config.json")
model = get_p2q(config, tokenizer, M_PATH, False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

accelerator = Accelerator()
model = accelerator.prepare(model)


RuntimeError: Error(s) in loading state_dict for P2QTransformer:
	size mismatch for encoder.positional_embedding: copying a param with shape torch.Size([4096, 768]) from checkpoint, the shape in current model is torch.Size([1024, 768]).
	size mismatch for decoder.positional_embedding: copying a param with shape torch.Size([4096, 768]) from checkpoint, the shape in current model is torch.Size([1024, 768]).

In [3]:
midi_path = "datasets/paired-dataset-5/performance/audio-https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DL2KdUIeYTQo.mid"
midi_dict = MidiDict.from_midi(midi_path)
tokenized_midi = tokenizer._tokenize_midi_dict(midi_dict=midi_dict)
enc_seq = []
for token in tokenized_midi:
    enc_seq.extend(tokenizer.encode([token]))

In [77]:
import torch
import torch.nn.functional as F

def beam_search(model, encoder_output, start_token, beam_width, max_length, end_token, device):
    sequences = [([start_token], 0.0)]  # Initialize with the start token and score 0.0
    
    for _ in range(max_length):
        all_candidates = []
        
        for seq, score in sequences:
            input_tensor = torch.tensor(seq, dtype=torch.long).unsqueeze(0).to(device)
            with torch.amp.autocast('cuda'):
                logits = model.forward(encoder_output, input_tensor)
            logits = logits[:, -1, :]  # Take the logits of the last token
            log_probs = F.log_softmax(logits, dim=-1)
            top_log_probs, top_indices = torch.topk(log_probs, beam_width)
            
            for i in range(beam_width):
                candidate = (seq + [top_indices[0][i].item()], score - top_log_probs[0][i].item())
                all_candidates.append(candidate)
        
        # Select the best beam_width candidates
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:beam_width]
        
        # Check if any sequence ends with the end_token
        for seq, score in sequences:
            if seq[-1] == end_token:
                return seq
            
    return sequences[0][0]

def encode_with_sliding_window(model, input_sequence, window_size, device):
    encoded_outputs = []

    for start in range(0, len(input_sequence), window_size):
        end = start + window_size
        chunk = input_sequence[start:end]
        if len(chunk) < window_size:
            # Padding the last chunk if it's smaller than the window size
            chunk = chunk + [0] * (window_size - len(chunk))
        
        chunk_tensor = torch.tensor(chunk, dtype=torch.long).unsqueeze(0).to(device)
        with torch.amp.autocast('cuda'):
            encoded_chunk = model.encode(chunk_tensor)
        encoded_outputs.append(encoded_chunk)

    # Concatenate all encoded chunks along the sequence dimension
    return torch.cat(encoded_outputs, dim=1)

def inference(sequence, model, beam_width, max_length, start_token, end_token, device, accelerator, window_size=4096):
    model.eval()
    
    # Encode input sequence
    with torch.amp.autocast('cuda'):
        encoder_output = encode_with_sliding_window(model, sequence, window_size, device)

    # Start beam search from the start token
    generated_sequence = beam_search(model, encoder_output, start_token, beam_width, max_length, end_token, device)
    return generated_sequence

# Example usage

# Convert start_token and end_token to scalar values if they are lists or tensors
start_token = tokenizer.encode('<S>')[0]  # Assuming tokenizer.encode returns a list
end_token = tokenizer.encode('<E>')[0]  # Assuming tokenizer.encode returns a list

beam_width = 3
max_length = 5000
window_size = 4096

# Assuming 'model' and 'accelerator' are defined and initialized properly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example sequence (should be tokenized appropriately for your model)

prediction = inference(
    sequence=enc_seq,
    model=model,
    beam_width=beam_width,
    max_length=max_length,
    start_token=start_token,
    end_token=end_token,
    device=device,
    accelerator=accelerator,
    window_size=window_size
)

print("Generated Sequence:", prediction)


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)